In [ ]:
import torch 
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# parallel processing
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True, nb_workers=16)
from tqdm import tqdm
tqdm.pandas()

# utility
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import os

In [ ]:
# loading the metrics
import evaluate
exact_match_metric = evaluate.load("exact_match")
sacrebleu = evaluate.load("sacrebleu")
meteor = evaluate.load("meteor")
rouge = evaluate.load("rouge")


def bleu_compute(pred, suffix):
    return sacrebleu.compute(predictions = pred, references = suffix)["score"]

def em_compute(pred, suffix):
    return exact_match_metric.compute(references=pred, predictions=suffix)['exact_match']

def meteor_compute(pred, suffix):
    return meteor.compute(predictions = pred, references = suffix)["meteor"]

def rouge_compute(pred, suffix):
    return rouge.compute(predictions = pred, references = suffix)["rougeL"]

To change depending on which model you want to attack!

In [3]:
checkpoint = "bigcode/starcoder2-3b"
# checkpoint = "bigcode/starcoder2-7b"
# checkpoint = "bigcode/starcoder2-15b"

# Load the sample datasets for the mem attacks (different deduplication rates)

- = 1 (df_1)
- = 2 (df_2)
- = 3 (df_3)
- \> 3 (df_g3)

Deduplication rate = 1

In [ ]:
df_1 = pd.read_parquet('mem-tune-replication_package/mem-tune/data/samples/fine-tune/mem-attack/fine-tune_attack_1.parquet')
df_1 = Dataset.from_pandas(df_1[['prefix_250', 'prefix_200', 'prefix_150', 'prefix_100', 'suffix']])

Deduplication rate = 2

In [ ]:
df_2 = pd.read_parquet('mem-tune-replication_package/mem-tune/data/samples/fine-tune/mem-attack/fine-tune_attack_2.parquet')
df_2 = Dataset.from_pandas(df_2[['prefix_250', 'prefix_200', 'prefix_150', 'prefix_100', 'suffix']])

Deduplication rate = 3

In [ ]:
df_3 = pd.read_parquet('mem-tune-replication_package/mem-tune/data/samples/fine-tune/mem-attack/fine-tune_attack_3.parquet')
df_3 = Dataset.from_pandas(df_3[['prefix_250', 'prefix_200', 'prefix_150', 'prefix_100', 'suffix']])

Deduplication rate > 3

In [ ]:
df_g3 = pd.read_parquet('mem-tune-replication_package/mem-tune/data/samples/fine-tune/mem-attack/fine-tune_attack_g3.parquet')
df_g3 = Dataset.from_pandas(df_g3[['prefix_250', 'prefix_200', 'prefix_150', 'prefix_100', 'suffix']])

Parameters for this experiment:
- training epochs fixed to 3
- prefix length fixed to 100 tokens
- We are gonna vay the deduplication rate

In [8]:
p_length = 'prefix_100'

Setup the column names:
- The generation will be the same 7b_ep0_d1, 7b__ep3_d2, 7b_ep_0_d3, 7b_ep3_dg3 (the epochs are fixed to three now so we don't need to keep track of that, but we will need to have also the baseline value ep_0)
- For the metrics we are gonna add the _metric, i.e. 7b_ep3_d3_bleu, 7b_ep3_d3_meteor, ...

In [9]:
# setup the experiment names

m_name = checkpoint.split('-')[1] # take only the nymber of parameters

epochs = ['_ep0', '_ep1', '_ep2', '_ep3']

deduplication = ['_d1', '_d2','_d3','_dg3']


# Load the models

In [10]:
# tokenizer
tokenizer = AutoTokenizer.from_pretrained(checkpoint, padding_side='left')
tokenizer.pad_token = tokenizer.eos_token

**IMPORTANT**

In the public replication package we will also share the fine-tuned models on Huggingface

In [11]:
# Model dirs
#To change depending on which model you want to attack!

# ep_dir = '/model_dir/scoder3b/epochs'
# ep1 = ep_dir +'/checkpoint-833'
# ep2 = ep_dir + '/checkpoint-1666'
# ep3= ep_dir + '/checkpoint-2499'

# ep_dir = '/model_dir/scoder7b/epochs'
# ep1 = ep_dir +'/checkpoint-833'
# ep2 = ep_dir + '/checkpoint-1666'
# ep3= ep_dir + '/checkpoint-2499'

ep_dir = '/model_dir/scoder15b/epochs'
ep1 = ep_dir +'/checkpoint-800'
ep2 = ep_dir + '/checkpoint-1600'
ep3= ep_dir + '/checkpoint-2400'


In [ ]:
# load the model

# base model
model = AutoModelForCausalLM.from_pretrained(checkpoint,device_map="auto")
epoch = epochs[0]

# model depending on the epoch
# model = AutoModelForCausalLM.from_pretrained(ep1,device_map="auto")
# epoch = epochs[1]

# model = AutoModelForCausalLM.from_pretrained(ep2,device_map="auto")
# epoch = epochs[2]

# model = AutoModelForCausalLM.from_pretrained(ep3,device_map="auto")
# epoch = epochs[3]

# eval mode
model.eval()

In [13]:
# define the pipeline
pipe = pipeline("text-generation", model = model, tokenizer = tokenizer, framework='pt', max_new_tokens=50)

In [ ]:
# IMPORTANT

gen_name_1 = m_name + epoch + deduplication[0]
gen_name_2 = m_name + epoch + deduplication[1]
gen_name_3 = m_name + epoch + deduplication[2]
gen_name_g3 = m_name + epoch + deduplication[3]

print(f"{gen_name_1}\n {gen_name_2}\n {gen_name_3}\n {gen_name_g3}")

# Now we can generate!

In [ ]:
gs_mem_1 = pipe(df_1[p_length], batch_size=32)
print('1 done')
gs_mem_2 = pipe(df_2[p_length], batch_size=32)
print('2 done')
gs_mem_3 = pipe(df_3[p_length], batch_size=32)
print('3 done')
gs_mem_g3 = pipe(df_g3[p_length], batch_size=32)
print('g3 done')

# Save the results on each of the datasets
df_1 = df_1.add_column(gen_name_1, [gs_mem_1[i][0]['generated_text'][len(df_1[p_length][i]):]  for i,_ in enumerate(df_1)])
df_2 = df_2.add_column(gen_name_2, [gs_mem_2[i][0]['generated_text'][len(df_2[p_length][i]):]  for i,_ in enumerate(df_2)])
df_3 = df_3.add_column(gen_name_3, [gs_mem_3[i][0]['generated_text'][len(df_3[p_length][i]):]  for i,_ in enumerate(df_3)])
df_g3 = df_g3.add_column(gen_name_g3, [gs_mem_g3[i][0]['generated_text'][len(df_g3[p_length][i]):]  for i,_ in enumerate(df_g3)])

# Evaluation

In [17]:
expath = 'mem-tune/evaluation/memorization/experiments/attack_mem_dedup'

In [18]:
# Initial save
df_eval_1= df_1.to_pandas()
df_eval_2= df_2.to_pandas()
df_eval_3= df_3.to_pandas()
df_eval_g3= df_g3.to_pandas()

# After is saved the first time
# df_eval_1 = pd.merge((pd.read_parquet(expath + '/attack_mem_dedup_1.parquet')), df_1.to_pandas()[gen_name_1], left_index =True, right_index=True)
# df_eval_2 = pd.merge((pd.read_parquet(expath + '/attack_mem_dedup_2.parquet')), df_2.to_pandas()[gen_name_2], left_index =True, right_index=True)
# df_eval_3 = pd.merge((pd.read_parquet(expath + '/attack_mem_dedup_3.parquet')), df_3.to_pandas()[gen_name_3], left_index =True, right_index=True)
# df_eval_g3 = pd.merge((pd.read_parquet(expath + '/attack_mem_dedup_g3.parquet')), df_g3.to_pandas()[gen_name_g3], left_index =True, right_index=True)

In [ ]:
em = gen_name_1 + '_em'
bleu = gen_name_1 + '_bleu'
met = gen_name_1 + '_meteor'
roug = gen_name_1 + '_rougeL'

print(f"{em}\n {bleu}\n {met}\n {roug}")

In [ ]:
# evaluation
df_eval_1[em] = df_eval_1.progress_apply(lambda x: em_compute( pred= [x[gen_name_1]], suffix = [x['suffix']]), axis=1)
df_eval_1[bleu] = df_eval_1.progress_apply(lambda x: bleu_compute( pred= [x[gen_name_1]], suffix = [x['suffix']]), axis=1)
df_eval_1[met] = df_eval_1.progress_apply(lambda x: meteor_compute( pred= [x[gen_name_1]], suffix = [x['suffix']]), axis=1)
df_eval_1[roug] = df_eval_1.progress_apply(lambda x: rouge_compute( pred= [x[gen_name_1]], suffix = [x['suffix']]), axis=1)

Deduplicaiton rate = 2

In [ ]:
em = gen_name_2 + '_em'
bleu = gen_name_2 + '_bleu'
met = gen_name_2 + '_meteor'
roug = gen_name_2 + '_rougeL'

print(f"{em}\n {bleu}\n {met}\n {roug}")

In [ ]:
# evaluation
df_eval_2[em] = df_eval_2.progress_apply(lambda x: em_compute( pred= [x[gen_name_2]], suffix = [x['suffix']]), axis=1)
df_eval_2[bleu] = df_eval_2.progress_apply(lambda x: bleu_compute( pred= [x[gen_name_2]], suffix = [x['suffix']]), axis=1)
df_eval_2[met] = df_eval_2.progress_apply(lambda x: meteor_compute( pred= [x[gen_name_2]], suffix = [x['suffix']]), axis=1)
df_eval_2[roug] = df_eval_2.progress_apply(lambda x: rouge_compute( pred= [x[gen_name_2]], suffix = [x['suffix']]), axis=1)

Deduplication rate = 3

In [ ]:
em = gen_name_3 + '_em'
bleu = gen_name_3 + '_bleu'
met = gen_name_3 + '_meteor'
roug = gen_name_3 + '_rougeL'

print(f"{em}\n {bleu}\n {met}\n {roug}")

In [ ]:
# evaluation
df_eval_3[em] = df_eval_3.progress_apply(lambda x: em_compute( pred= [x[gen_name_3]], suffix = [x['suffix']]), axis=1)
df_eval_3[bleu] = df_eval_3.progress_apply(lambda x: bleu_compute( pred= [x[gen_name_3]], suffix = [x['suffix']]), axis=1)
df_eval_3[met] = df_eval_3.progress_apply(lambda x: meteor_compute( pred= [x[gen_name_3]], suffix = [x['suffix']]), axis=1)
df_eval_3[roug] = df_eval_3.progress_apply(lambda x: rouge_compute( pred= [x[gen_name_3]], suffix = [x['suffix']]), axis=1)

Deduplication rate > 3

In [ ]:
em = gen_name_g3 + '_em'
bleu = gen_name_g3 + '_bleu'
met = gen_name_g3 + '_meteor'
roug = gen_name_g3 + '_rougeL'

print(f"{em}\n {bleu}\n {met}\n {roug}")

In [ ]:
# evaluation
df_eval_g3[em] = df_eval_g3.progress_apply(lambda x: em_compute( pred= [x[gen_name_g3]], suffix = [x['suffix']]), axis=1)
df_eval_g3[bleu] = df_eval_g3.progress_apply(lambda x: bleu_compute( pred= [x[gen_name_g3]], suffix = [x['suffix']]), axis=1)
df_eval_g3[met] = df_eval_g3.progress_apply(lambda x: meteor_compute( pred= [x[gen_name_g3]], suffix = [x['suffix']]), axis=1)
df_eval_g3[roug] = df_eval_g3.progress_apply(lambda x: rouge_compute( pred= [x[gen_name_g3]], suffix = [x['suffix']]), axis=1)

# Save all the results

In [29]:
expath = 'mem-tune/evaluation/memorization/experiments/attack_mem_dedup'

In [30]:
df_eval_1.to_parquet(os.path.join(expath, 'attack_mem_dedup_1.parquet'))
df_eval_2.to_parquet(os.path.join(expath, 'attack_mem_dedup_2.parquet'))
df_eval_3.to_parquet(os.path.join(expath, 'attack_mem_dedup_3.parquet'))
df_eval_g3.to_parquet(os.path.join(expath, 'attack_mem_dedup_g3.parquet'))